In [1]:
from Predictability import *

In [2]:
get_predictability([10,20,10,20,10,20,10,20,10,20])

0.7428913814

## Entropy 

In [ ]:
Real_Entropy([1]*5), Real_Entropy([1]*50), Real_Entropy(list(np.random.randint(0, 10, 10)))

## Predictability

In [3]:
print(1)

1


In [4]:
from predictability.Predictability import *
import os
import numpy as np
import pandas as pd
from util.load_data import *
data_path=r'D:\MyWorks\糖尿病预测\blood_glucose_prediction'
# data_type='ohio_data'
data_type='simulator_data'
# data_type='hospital_data'
# data_type='EastT1DM'

def get_pre(data_type,is_int=False):
    df=pd.DataFrame(columns=('patient','predictability'))
    mean_predictability=[]
    dataset=f'{data_path}/data/{data_type}'
    if data_type=='ohio_data':
        for patient in [540, 544, 552, 567, 584, 596,559,563,570,575,588,591]:
            print(patient)
            train_path = f'{dataset}/{patient}-ws-training.csv'
            test_path = f'{dataset}/{patient}-ws-testing.csv'
            train_data = load_data(train_path, 'csv')['gl_value'].reset_index(drop=True)
            test_data = load_data(test_path, 'csv')['gl_value'].reset_index(drop=True)
            train_data = train_data.dropna(axis=0)
            test_data = test_data.dropna(axis=0)
            data=train_data.append(test_data)
            print(data)
            if is_int:
                data=data.astype(int)
            pre=get_predictability(data)
            series=pd.Series({'patient':patient,'predictability':pre})
            df=df.append(series,ignore_index=True)
            mean_predictability.append(pre)
    else:
        for patient in os.listdir(dataset):
            print(patient)
            patient, suffix = patient.split('.')
            data = load_data(dataset + f'/{patient}.{suffix}', suffix).iloc[:,0].dropna()
            if is_int:
                data=data.astype(int)
            pre=get_predictability(data)
            mean_predictability.append(pre)
            series=pd.Series({'patient':patient,'predictability':pre})
            print(series)
            df=df.append(series,ignore_index=True)
        # # data = data['CGM']
        # # modelset = ['LR', 'Ridge', 'LASSO', 'RF', 'KNN']
        # 
        # for modeltype in tqdm(modelset):
        #     onestep_train(modeltype, data_type, patient, time_interval, time_step,suffix=suffix)
    save_path=f'{data_path}/predictability/result/{data_type}.csv'
    if is_int:
        save_path=f'{data_path}/predictability/result/{data_type}_int.csv'
    print(df)
    df.to_csv(save_path,encoding='utf-8-sig')
    return np.mean(mean_predictability)
res=get_pre('ohio_data',is_int=True)
# res=get_pre('simulator_data',is_int=True)

res=get_pre('hospital_data',is_int=True)
res=get_pre('EastT1DM',is_int=True)

print(res)

540
0        76.0
1        76.0
2        76.0
3        76.0
4        76.0
        ...  
3060    225.0
3061    240.0
3062    249.0
3063    254.0
3064    248.0
Name: gl_value, Length: 16312, dtype: float64
544
0       129.0
1       128.0
2       129.0
3       131.0
4       133.0
        ...  
3230    262.0
3231    262.0
3232    262.0
3233    262.0
3234    262.0
Name: gl_value, Length: 15906, dtype: float64
552
0        95.0
1        95.0
2        95.0
3        95.0
4        95.0
        ...  
3997    215.0
3998    215.0
3999    215.0
4000    215.0
4001    215.0
Name: gl_value, Length: 15233, dtype: float64
567
0        72.0
1        71.0
2        71.0
3        71.0
4        71.0
        ...  
2973    157.0
2974    157.0
2975    157.0
2976    157.0
2977    157.0
Name: gl_value, Length: 16513, dtype: float64
584
0       48.0
1       48.0
2       53.0
3       63.0
4       69.0
        ... 
2989    77.0
2990    71.0
2991    74.0
2992    72.0
2993    78.0
Name: gl_value, Length: 16241, dtype:

In [246]:
import pandas as pd
import matplotlib.pyplot as plt
import joblib
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['SimHei']

# data_type='ohio_data'
data_type='simulator_data'
# data_type='hospital_data'
metric='rmse'
time_hotizon=30
# def get_predict_metric(data_type='simulator_data',metric='rmse',time_hotizon=60,time_interval=5):
#     获取one_data的GRU结果
#     source_path=r'D:\MyWorks\糖尿病预测\blood_glucose_prediction\predictability\result'
#     data_path=f'{source_path}/{data_type}_int.csv'
#     data=pd.read_csv(data_path,index_col=0,converters={'patient':str} )
#     result_path=r'D:\MyWorks\糖尿病预测\blood_glucose_prediction\result\multi_step'
#     result_data_path=f'{result_path}/{data_type}_epoch_1000'
#     gru_data=None
#     for patient in os.listdir(result_data_path):
#         gru_path=f'{result_data_path}/{patient}/GRU_1.csv'
#         gru=pd.read_csv(gru_path)
#         gru['patient']=patient
#         gru['patient']=gru['patient'].astype(object)
#         gru.rename(columns={'Unnamed: 0':"metric"},inplace=True)
#         gru=gru[gru['metric']==metric]
#         if gru_data is None:
#             gru_data=gru
#         else:
#             gru_data=gru_data.append(gru)
#     data=pd.merge(data,gru_data,how='left',on='patient')
#     # print(data)
#     data=data.sort_values('predictability',ascending=True).reset_index(drop=True)
#     
#     fig=plt.figure(figsize=(8,6))
#     ax1=fig.add_subplot(111)
#     data['predictability'].plot(ax=ax1,style='b',label='predictability')
#     ax1.set_ylabel("可预测性")
#     # ax1.set_yticks(np.arange(0.8,0.9,0.01))
#     plt.legend(loc=2)
#     plt.xlabel('根据可预测性排序的模拟器患者')
#     
#     ax2=ax1.twinx()
#     data[str(time_hotizon//time_interval-1)].plot(ax=ax2,style='r',label='30 minutes')
#     # ax2.set_yticks(np.arange(13,35,2))
#     ax2.set_ylabel("RMSE")
#     plt.legend(loc=1)
#     
#     # plt.grid()
#     plt.title(f'{time_hotizon}分钟的{metric}比较')
#     plt.savefig(f'figure/{data_type}_{metric}_{time_hotizon}.png',dpi=400,bbox_inches='tight')
    
# def get_predict_metric(data_type='simulator_data',metric='rmse',time_hotizon=60,time_interval=5):
#     # 获取all_data的GRU结果
#     source_path=r'D:\MyWorks\糖尿病预测\blood_glucose_prediction\predictability\result'
#     data_path=f'{source_path}/{data_type}_int.csv'
#     data=pd.read_csv(data_path,index_col=0,converters={'patient':str} )
#     result_path=r'D:\MyWorks\糖尿病预测\blood_glucose_prediction\result\multi_step_all_data'
#     result_data_path=f'{result_path}/{data_type}_epoch_300'
#     gru_data=None
#     for patient in os.listdir(result_data_path):
#         gru_path=f'{result_data_path}/{patient}/GRU.csv'
#         gru=pd.read_csv(gru_path)
#         gru['patient']=patient
#         gru['patient']=gru['patient'].astype(object)
#         gru.rename(columns={'Unnamed: 0':"metric"},inplace=True)
#         gru=gru[gru['metric']==metric]
#         if gru_data is None:
#             gru_data=gru
#         else:
#             gru_data=gru_data.append(gru)
#     data=pd.merge(data,gru_data,how='left',on='patient')
#     # print(data)
#     data=data.sort_values('predictability',ascending=True).reset_index(drop=True)
#     
#     fig=plt.figure(figsize=(8,6))
#     ax1=fig.add_subplot(111)
#     data['predictability'].plot(ax=ax1,style='b',label='predictability')
#     ax1.set_ylabel("可预测性")
#     # ax1.set_yticks(np.arange(0.8,0.9,0.01))
#     plt.legend(loc=2)
#     plt.xlabel('根据可预测性排序的模拟器患者')
#     
#     ax2=ax1.twinx()
#     data[str(time_hotizon//time_interval-1)].plot(ax=ax2,style='r',label='30 minutes')
#     # ax2.set_yticks(np.arange(13,35,2))
#     ax2.set_ylabel(metric)
#     plt.legend(loc=1)
#     
#     # plt.grid()
#     plt.title(f'{time_hotizon}分钟的{metric}比较')
#     plt.savefig(f'figure/all_data/{data_type}_{metric}_{time_hotizon}.png',dpi=400,bbox_inches='tight')

def get_predict_metric(data_type='simulator_data',metric='rmse',time_hotizon=60,time_interval=5,model_type='GRU'):
    source_path=r'D:\MyWorks\糖尿病预测\blood_glucose_prediction\predictability\result'
    data_path=f'{source_path}/{data_type}_int.csv'
    data=pd.read_csv(data_path,index_col=0,converters={'patient':str} )
    result_path=r'D:\MyWorks\糖尿病预测\blood_glucose_prediction\result\one_step'
    # result_data_path=f'{result_path}/{data_type}_30'
    result_data_path=f'{result_path}/{data_type}_epoch_1000'
    
    gru_data=None
    for patient in os.listdir(result_data_path):
        gru_path=f'{result_data_path}/{patient}/{model_type}.pkl'
        gru=joblib.load(gru_path)[f'{time_hotizon}minuts']
        gru=pd.DataFrame({'patient':[patient],metric:[gru[4]]})
        # gru['patient']=patient
        # gru['patient']=gru['patient'].astype(object)
        # gru.rename(columns={'Unnamed: 0':"metric"},inplace=True)
        # gru=gru[gru['metric']==metric]
        # print(gru)
        if gru_data is None:
            gru_data=gru
        else:
            gru_data=gru_data.append(gru)
    data=pd.merge(data,gru_data,how='left',on='patient')
    # print(data)
    data=data.sort_values('predictability',ascending=True).reset_index(drop=True)
    data['predictability']=1-data['predictability']
    print(data)
    fig=plt.figure(figsize=(8,6))
    ax1=fig.add_subplot(111)
    data['predictability'].plot(ax=ax1,style='b',label='predictability')
    ax1.set_ylabel("可预测性")
    # ax1.set_yticks(np.arange(0.8,0.9,0.01))
    plt.legend(loc=2)
    plt.xlabel('根据可预测性排序的模拟器患者')
    
    ax2=ax1.twinx()
    data[metric].plot(ax=ax2,style='r',label='30 minutes')
    # ax2.set_yticks(np.arange(13,35,2))
    ax2.set_ylabel(metric)
    plt.legend(loc=1)
    
    # plt.grid()
    plt.title(f'{time_hotizon}分钟的{metric}比较-{model_type}')
    plt.savefig(f'figure/one_data/{model_type}/{data_type}_{metric}_{time_hotizon}.png',dpi=400,bbox_inches='tight')

In [278]:
def get_multi_model(data_type='simulator_data',metric='rmse',time_hotizon=30,time_interval=5):
    source_path=r'D:\MyWorks\糖尿病预测\blood_glucose_prediction\predictability\result'
    data_path=f'{source_path}/{data_type}_int.csv'
    data=pd.read_csv(data_path,index_col=0,converters={'patient':str} )
    data['predictability']=1-data['predictability']
    data=data.sort_values('predictability',ascending=False).reset_index(drop=True)

    result_path=r'D:\MyWorks\糖尿病预测\blood_glucose_prediction\result\one_step'
    result_data_path=f'{result_path}/{data_type}_30'
    ridge_data=None
    svr_data=None
    RF_data=None
    gru_data=None
    for patient in os.listdir(result_data_path):
        gru_path=f'{result_data_path}/{patient}/Ridge.pkl'
        gru=joblib.load(gru_path)
        gru=pd.DataFrame({'patient':[patient],metric:[gru[4][1]/100]})
        if ridge_data is None:
            ridge_data=gru
        else:
            ridge_data=ridge_data.append(gru)
    for patient in os.listdir(result_data_path):
        gru_path=f'{result_data_path}/{patient}/SVR.pkl'
        gru=joblib.load(gru_path)
        gru=pd.DataFrame({'patient':[patient],metric:[gru[4][1]/100]})
        if svr_data is None:
            svr_data=gru
        else:
            svr_data=svr_data.append(gru)  
    for patient in os.listdir(result_data_path):
        gru_path=f'{result_data_path}/{patient}/RF.pkl'
        gru=joblib.load(gru_path)
        gru=pd.DataFrame({'patient':[patient],metric:[gru[4][1]/100]})
        if RF_data is None:
            RF_data=gru
        else:
            RF_data=RF_data.append(gru)
            
    result_path=r'D:\MyWorks\糖尿病预测\blood_glucose_prediction\result\multi_step'
    result_data_path=f'{result_path}/{data_type}_epoch_1000'
    for patient in os.listdir(result_data_path):
        gru_path=f'{result_data_path}/{patient}/GRU.pkl'
        gru=joblib.load(gru_path)
        gru=pd.DataFrame({'patient':[patient],metric:[gru[4][1]/100]})
        if gru_data is None:
            gru_data=gru
        else:
            gru_data=gru_data.append(gru)
            
    merge_gru=pd.merge(data,gru_data,how='left',on='patient')
    merge_ridge=pd.merge(data,ridge_data,how='left',on='patient')
    merge_svr=pd.merge(data,svr_data,how='left',on='patient')
    merge_rf=pd.merge(data,RF_data,how='left',on='patient')
    print(merge_gru)
    print(merge_ridge)
    fig=plt.figure(figsize=(8,6))
    ax1=fig.add_subplot(111)
    data['predictability'].plot(ax=ax1,style='r--',label='predictability',lw=3)
    ax1.set_ylabel("可预测性")
    # ax1.set_yticks(np.arange(0.8,0.9,0.01))
    plt.legend(loc=2)
    plt.xlabel('根据可预测性排序的模拟器患者')
    
    # ax2=ax1.twinx()
    merge_ridge['smape'].plot(ax=ax1,style='b',label='ridge',lw=3)
    merge_gru['smape'].plot(ax=ax1,label='gru',lw=3)
    merge_svr['smape'].plot(ax=ax1,label='SVR',lw=3)
    merge_rf['smape'].plot(ax=ax1,label='RF',lw=3)
    # ax2.set_yticks(np.arange(13,35,2))
    # ax2.set_ylabel(metric)
    plt.legend(loc=1)
    
    plt.grid()
    plt.title(f'{time_hotizon}分钟的{metric}比较-{data_type}')
    plt.savefig(f'figure/compare_figure/{data_type}_{metric}_{time_hotizon}.png',dpi=400,bbox_inches='tight')

In [247]:
metric='smape'
# get_predict_metric('simulator_data',metric,5)
get_predict_metric('simulator_data',metric,30,model_type='GRU')
# get_predict_metric('simulator_data',metric,60)
# get_predict_metric('ohio_data',metric,5)
get_predict_metric('ohio_data',metric,30,model_type='GRU')
# get_predict_metric('ohio_data',metric,60)
# get_predict_metric('hospital_data',metric,15,15)
get_predict_metric('hospital_data',metric,30,15,model_type='GRU')
# get_predict_metric('hospital_data',metric,60,15)

                             patient  predictability     smape
0        child#004_simulation_8weeks        0.175650  5.279063
1        child#005_simulation_8weeks        0.167038  4.927157
2        child#001_simulation_8weeks        0.159518  8.217657
3        adult#003_simulation_8weeks        0.156791  6.579209
4        child#002_simulation_8weeks        0.149233  4.861039
5        adult#006_simulation_8weeks        0.146556  5.010334
6   adolescent#002_simulation_8weeks        0.144547  7.342054
7   adolescent#003_simulation_8weeks        0.144062  4.986194
8   adolescent#006_simulation_8weeks        0.143994  6.095695
9        adult#004_simulation_8weeks        0.140785  5.268201
10       child#010_simulation_8weeks        0.139963  6.704653
11  adolescent#004_simulation_8weeks        0.138492  5.451986
12  adolescent#005_simulation_8weeks        0.136931  5.120737
13  adolescent#008_simulation_8weeks        0.135907  6.685163
14       child#003_simulation_8weeks        0.131883  6

In [279]:
# get_multi_model('simulator_data',metric,5)
get_multi_model('simulator_data',metric,30)
# get_multi_model('simulator_data',metric,60)
# get_multi_model('ohio_data',metric,5)
get_multi_model('ohio_data',metric,30)
# get_multi_model('ohio_data',metric,60)
# get_multi_model('hospital_data',metric,15,15)
get_multi_model('hospital_data',metric,30,15)
# get_multi_model('hospital_data',metric,60,15)


                             patient  predictability     smape
0        child#004_simulation_8weeks        0.175650  0.020004
1        child#005_simulation_8weeks        0.167038  0.019624
2        child#001_simulation_8weeks        0.159518  0.038671
3        adult#003_simulation_8weeks        0.156791  0.028449
4        child#002_simulation_8weeks        0.149233  0.017845
5        adult#006_simulation_8weeks        0.146556  0.020043
6   adolescent#002_simulation_8weeks        0.144547  0.031206
7   adolescent#003_simulation_8weeks        0.144062  0.017136
8   adolescent#006_simulation_8weeks        0.143994  0.025756
9        adult#004_simulation_8weeks        0.140785  0.020472
10       child#010_simulation_8weeks        0.139963  0.027306
11  adolescent#004_simulation_8weeks        0.138492  0.023091
12  adolescent#005_simulation_8weeks        0.136931  0.019393
13  adolescent#008_simulation_8weeks        0.135907  0.025697
14       child#003_simulation_8weeks        0.131883  0

In [6]:
import os
import pandas as pd
for d in os.listdir('result'):
    data=pd.read_csv(f'result/{d}',index_col=0)['predictability']
    print(d,data.describe())




EastT1DM_int.csv count    12.000000
mean      0.691882
std       0.018849
min       0.667403
25%       0.677950
50%       0.689370
75%       0.707486
max       0.721103
Name: predictability, dtype: float64
hospital_data_int.csv count    91.000000
mean      0.706269
std       0.028742
min       0.653962
25%       0.685990
50%       0.702497
75%       0.717520
max       0.861208
Name: predictability, dtype: float64
ohio_data.csv count       0
unique      0
top       NaN
freq      NaN
Name: predictability, dtype: object
ohio_data_int.csv count    12.000000
mean      0.889272
std       0.070086
min       0.803418
25%       0.830054
50%       0.870501
75%       0.964096
max       0.986480
Name: predictability, dtype: float64
simulator_data_int.csv count    30.000000
mean      0.863380
std       0.014155
min       0.824350
25%       0.855955
50%       0.868570
75%       0.874556
max       0.879416
Name: predictability, dtype: float64
